In [1]:
from sciterra import Atlas
from sciterra.vectorization import vectorizers

import sys
sys.path.append("/Users/nathanielimel/uci/projects/citesim/src")

from analysis.plot import atlas_to_measurements

IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html


In [2]:
atlas_dirs = {
    "Physics": "/Users/nathanielimel/uci/projects/citesim/outputs/librarian=S2/vectorizer=SciBERT/center=hafenLowredshiftLymanLimit2017",
    "Linguistics": "/Users/nathanielimel/uci/projects/citesim/outputs/librarian=S2/vectorizer=SciBERT/center=Imeletal2022",
    "Medicine": "/Users/nathanielimel/uci/projects/citesim/outputs/librarian=S2/vectorizer=SciBERT/center=Torres2013",
    "Education": "/Users/nathanielimel/uci/projects/citesim/outputs/librarian=S2/vectorizer=SciBERT/center=Ololube2012",
}

In [3]:
# Load the source atlas for the field
field = "Physics"
atl = Atlas.load(atlas_dirs[field])
len(atl)

30385

In [4]:
from sciterra import Cartographer
def copy_atlas(atl: Atlas, vectorizer: str, **kwargs) -> Atlas:
    """Create a new atlas containing the publications of an old one, and having a projection with a different vectorizer."""

    atl_new = Atlas(
        publications=list(atl.publications.values()),
    )
    crt = Cartographer(vectorizer=vectorizers[vectorizer](device="mps"))
    atl_new = crt.project(atl_new, **kwargs)
    return atl_new

In [5]:
atl_new: Atlas = copy_atlas(atl, "GPT2")
atl_new.projection

Using device: mps.


embedding documents: 30392it [1:49:45,  4.61it/s]                             


In [7]:
# save to appropriate dir for analysis

# This is our 'all data' csv

# save_dir = "/Users/nathanielimel/uci/projects/citesim/outputs/librarian=S2/vectorizer=SciBERT/center=hafenLowredshiftLymanLimit2017".replace("vectorizer=SciBERT", "vectorizer=GPT2")

# save_dir = "/Users/nathanielimel/uci/projects/citesim/outputs/librarian=S2/vectorizer=SciBERT/center=Imeletal2022".replace("vectorizer=SciBERT", "vectorizer=GPT2")

# save_dir = "/Users/nathanielimel/uci/projects/citesim/outputs/librarian=S2/vectorizer=SciBERT/center=Torres2013".replace("vectorizer=SciBERT", "vectorizer=GPT2")

# save_dir = "/Users/nathanielimel/uci/projects/citesim/outputs/librarian=S2/vectorizer=SciBERT/center=Ololube2012".replace("vectorizer=SciBERT", "vectorizer=GPT2")

save_dir = atlas_dirs[field].replace("vectorizer=SciBERT", "vectorizer=GPT2")

save_dir

'/Users/nathanielimel/uci/projects/citesim/outputs/librarian=S2/vectorizer=GPT2/center=hafenLowredshiftLymanLimit2017'

In [10]:
# save the atlas
# TODO: create an atlas.copy or from_atlas classmethod
atl_new.center = atl.center
atl_new.history = atl.history
atl_new.save(save_dir)


/Users/nathanielimel/uci/projects/sciterra/src/sciterra/mapping/atlas.py:114: UserWarning: Overwriting existing file at /Users/nathanielimel/uci/projects/citesim/outputs/librarian=S2/vectorizer=GPT2/center=hafenLowredshiftLymanLimit2017/publications.pkl.
/Users/nathanielimel/uci/projects/sciterra/src/sciterra/mapping/atlas.py:114: UserWarning: Overwriting existing file at /Users/nathanielimel/uci/projects/citesim/outputs/librarian=S2/vectorizer=GPT2/center=hafenLowredshiftLymanLimit2017/projection.pkl.
/Users/nathanielimel/uci/projects/sciterra/src/sciterra/mapping/atlas.py:114: UserWarning: Overwriting existing file at /Users/nathanielimel/uci/projects/citesim/outputs/librarian=S2/vectorizer=GPT2/center=hafenLowredshiftLymanLimit2017/bad_ids.pkl.
/Users/nathanielimel/uci/projects/sciterra/src/sciterra/mapping/atlas.py:116: UserWarning: Writing to /Users/nathanielimel/uci/projects/citesim/outputs/librarian=S2/vectorizer=GPT2/center=hafenLowredshiftLymanLimit2017/history.pkl.
/Users/nat

In [11]:
# measure with diff vectorizer

vectorizer = "GPT2"

df = atlas_to_measurements(
    atl_new,
    vectorizer=vectorizers[vectorizer](device="mps"),
    fields_of_study=[field],
)
df

# Okay, this is tricky; we're going to need to keep the identity of the papers when vectorizing; perhaps just by returning a column of identifiers.

Using device: mps.
Between update 64 and the final update (66) there were 1334 publications added to the Atlas.
Convergence criterion dict_items([('num_pubs_added', 1000), ('kernel_size', 16)]) (=> index 64 out of 67 total updates) yields 14625 ids out of 30385 total ids.
computing cosine similarity for 14625 embeddings with batch size 1000.


100%|██████████| 15/15 [00:05<00:00,  2.81it/s]


Computing ['density', 'edginess'] for 14625 publications.


100%|██████████| 14625/14625 [00:24<00:00, 585.75it/s]


There are 14605 total observations after filtering.


,density,edginess,citations_per_year,is_center
0,1058.675939,0.977032,14.000000,False
1,1545.550332,0.814415,1.894737,False
2,186.395648,0.505732,21.714286,False
3,1501.911690,0.594145,1.500000,False
4,1846.261870,0.452021,3.363636,False
...,...,...,...,...
14620,1550.733908,0.644650,6.583333,False
14621,2126.263669,0.423650,7.062500,False
14622,2112.960384,0.694545,21.307692,False
14623,1581.126069,0.870435,4.333333,False


In [ ]:
import os

save_fn = "all_data.csv"
save_fn = os.path.join(save_dir, save_fn)

df.to_csv(save_fn, index=False)

In [ ]:
# Now do it for the other settings, and refactor in the other notebook in order to compare

# will we combine fields? If so, should probably only do so at the level of z-scored data.